In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
df = gpd.read_file('data/All_Roads/HWY_All_Roads.shp')
df.head()

,OBJECTID,RouteId,LRSFromDat,LRSToDate,GlobalID,Shape_Leng,geometry
0,2,ALA_ALA_11TH ST_P,2019-10-14,NaN,{98D954CB-3CBD-448D-894D-B6719349DB7A},0.005911,"LINESTRING Z (-122.29284 37.77362 0.00000, -12..."
1,3,ALA_ALA_1ST ST_P,2019-10-14,NaN,{9770BDFA-5BBB-4F18-96AE-0AE1093674F3},0.007187,"LINESTRING Z (-122.30876 37.78186 0.00000, -12..."
2,4,ALA_ALA_2ND ST_P,2019-10-14,NaN,{F936CD65-EAA7-4C92-9346-3068A7A424A7},0.008873,MULTILINESTRING Z ((-122.29008 37.77652 0.0000...
3,5,ALA_ALA_3RD ST_P,2019-10-14,NaN,{57ACC169-F926-49A0-BE82-7E883CB1A0F1},0.015814,MULTILINESTRING Z ((-122.28851 37.77085 0.0000...
4,6,ALA_ALA_4TH ST_P,2019-10-14,NaN,{55D0F220-98DF-44AB-8938-F08B4B86DA84},0.006718,MULTILINESTRING Z ((-122.28602 37.77261 0.0000...


In [3]:
df = df.set_geometry('geometry')

In [4]:
bears = pd.read_csv('data/inat/inat_04252023_counties.csv')
bears = bears[['latitude', 'longitude', 'observed_on', 'county']]
bears['dist_from_road'] = ''

In [5]:
bears.head()

,latitude,longitude,observed_on,county,dist_from_road
0,36.560760,-118.767601,2006-06-27,Tulare,
1,37.770596,-119.510771,2007-09-07,Mariposa,
2,37.770596,-119.510771,2009-06-14,Mariposa,
3,37.665342,-119.724884,2007-09-22,Mariposa,
4,39.432481,-122.974434,2011-08-12,Lake,


In [15]:
from tqdm import tqdm

In [16]:
for i in tqdm(range(bears.shape[0])):
    bear_point = gpd.points_from_xy([bears.iloc[i,0]], [bears.iloc[i,1]])
    dist = 1000000
    for j in range(df.shape[0]):
        dist = min(dist, df.iloc[j,-1].distance(bear_point[0]))
    bears.iloc[i,-1] = dist * 0.000621 # convert to miles
    # bears.iloc[i,-1] = dist

100%|████████████████████████████████████| 5972/5972 [63:32:58<00:00, 38.31s/it]


In [17]:
bears.head()

,latitude,longitude,observed_on,county,dist_from_road
0,36.560760,-118.767601,2006-06-27,Tulare,0.132884
1,37.770596,-119.510771,2007-09-07,Mariposa,0.133741
2,37.770596,-119.510771,2009-06-14,Mariposa,0.133741
3,37.665342,-119.724884,2007-09-22,Mariposa,0.133789
4,39.432481,-122.974434,2011-08-12,Lake,0.135994


In [18]:
bears.to_csv('withwater.csv', index=False)

end

In [19]:
df_bears_f = pd.read_csv('data/inat/inat_04252023_final.csv')
df_bears_f.head(10)

,observed_on,latitude,longitude,county,dist_from_water,dist_from_campsite,landcode
0,2006-06-27,36.560760,-118.767601,Tulare,43,0.133035,52
1,2007-09-07,37.770596,-119.510771,Mariposa,185,0.133892,52
2,2009-06-14,37.770596,-119.510771,Mariposa,185,0.133892,52
3,2007-09-22,37.665342,-119.724884,Mariposa,185,0.133940,42
4,2011-08-12,39.432481,-122.974434,Lake,201,0.136146,52
5,2007-04-04,41.732258,-124.076564,Del Norte,273,0.137637,42
6,2011-11-08,38.576470,-122.710560,Sonoma,201,0.135656,52
7,2003-06-14,36.596700,-118.716330,Tulare,43,0.133028,42
8,2009-04-30,37.732643,-119.560681,Mariposa,185,0.133898,52
9,2009-05-16,37.980401,-119.791609,Tuolumne,19,0.134108,42


In [20]:
df_bears_f.shape

(5972, 7)

In [21]:
bears.shape

(5972, 5)

In [23]:
df_bears_f['dist_from_road'] = bears['dist_from_road']
df_bears_f.head(20)

,observed_on,latitude,longitude,county,dist_from_water,dist_from_campsite,landcode,dist_from_road
0,2006-06-27,36.560760,-118.767601,Tulare,43,0.133035,52,0.132884
1,2007-09-07,37.770596,-119.510771,Mariposa,185,0.133892,52,0.133741
2,2009-06-14,37.770596,-119.510771,Mariposa,185,0.133892,52,0.133741
3,2007-09-22,37.665342,-119.724884,Mariposa,185,0.133940,42,0.133789
4,2011-08-12,39.432481,-122.974434,Lake,201,0.136146,52,0.135994
5,2007-04-04,41.732258,-124.076564,Del Norte,273,0.137637,42,0.137486
6,2011-11-08,38.576470,-122.710560,Sonoma,201,0.135656,52,0.135504
7,2003-06-14,36.596700,-118.716330,Tulare,43,0.133028,42,0.132877
8,2009-04-30,37.732643,-119.560681,Mariposa,185,0.133898,52,0.133746
9,2009-05-16,37.980401,-119.791609,Tuolumne,19,0.134108,42,0.133957


In [24]:
df_bears_f.to_csv('data/inat/inat_04252023_final.csv', index=False)